# Table 2 Databases 
### (with option of downloading only single files)

In [195]:
import bs4
from bs4 import BeautifulSoup as bs
#from tqdm import tqdm
import requests
from selenium import webdriver
import pandas as pd

http://split4.pmfst.hr/dadp


In [197]:
url = 'http://split4.pmfst.hr/dadp/?a=list'
request = requests.get(url)
soup = bs(request.text)


In [198]:
#display = soup.select(".display")[0]
#head = display.find_all('thead')[0]
content = soup.find_all("table")[1].find_all('th')
heads = [h.contents[0] for h in content]
heads


['DADP ID',
 'Entry',
 'Uniprot code',
 'Species',
 'Sequence length',
 'Signal sequence',
 'Bioactive sequence']

In [199]:
content = soup.find_all("table")[1].find_all('tbody')
content
# is dynamic table

[]

In [200]:
import time

url = 'http://split4.pmfst.hr/dadp/?a=list'
#driver = webdriver.Chrome(executable_path='/Applications/Google Chrome.app') 
driver = webdriver.Chrome()
driver.get(url)
time.sleep(15)
html = driver.page_source

In [201]:
soup = bs(html, "html.parser")
table = soup.find_all('table')[1]# [0] is top bar
table_rows = table.find_all('tr')

In [41]:
els = table_rows[2].find_all('td')
cont = [e.contents[0] for e in els]
cont

[<a href='./?a=kartica&amp;id=&lt;a href="./?a=kartica&amp;id=SP_P86817"&gt;SP_P86817&lt;/a&gt;'></a>,
 '[Ala1,Thr6,Asp11]-phyllokinin',
 'P86817',
 'Physalaemus signifer',
 '11',
 '/',
 'APPGFTPFRID']

In [204]:
data = {}
i = 0
for r in table_rows[1:]:# header in row 1
    els = r.find_all('td')
    cont = [e.contents[0] if len(e)>0 else 'None' for e in els]
    id = 'SP_'+cont[2]
    data[id] = [id]+cont
    i+=1
print(i)

df = pd.DataFrame.from_dict(data, orient='index')
df.columns = ['ID']+heads
df.head(2)


2571


,ID,DADP ID,Entry,Uniprot code,Species,Sequence length,Signal sequence,Bioactive sequence
SP_P86816,SP_P86816,[],"[Ala1,Thr6,Ala11]-phyllokinin",P86816,Physalaemus signifer,11,/,APPGFTPFRIA
SP_P86817,SP_P86817,[],"[Ala1,Thr6,Asp11]-phyllokinin",P86817,Physalaemus signifer,11,/,APPGFTPFRID


In [207]:
dfnew = df[['Uniprot code', 'Entry', 'Species', 'Bioactive sequence' ]]

dfnew.insert(2, 'AB', [1 for i in range(dfnew.shape[0]) ] )
dfnew.insert(3, 'description', ['None' for i in range(dfnew.shape[0]) ])
dfnew.insert(5, 'dataset', ['DADP.html' for i in range(dfnew.shape[0]) ] )

dfnew.columns = newcolnames = ['id', 'name', 'AB', 'description', 'OX', 'dataset', 'seq']
dfnew.to_csv('DADP_out.csv', encoding='utf-8')

dfnew.head(2)

,id,name,AB,description,OX,dataset,seq
SP_P86816,P86816,"[Ala1,Thr6,Ala11]-phyllokinin",1,None,Physalaemus signifer,DADP.html,APPGFTPFRIA
SP_P86817,P86817,"[Ala1,Thr6,Asp11]-phyllokinin",1,None,Physalaemus signifer,DADP.html,APPGFTPFRID


https://peptaibol.cryst.bbk.ac.uk/introduction.htm

In [208]:
url = 'https://peptaibol.cryst.bbk.ac.uk/peptaibol_database_1lettercodes.htm'
# request = requests.get(url) not working because page not save
# save tables outer html to 'peptaibol_outer_table.txt'
f = open('peptaibol_outer_table.txt', 'r')
soup = bs(f, "html.parser")

names = [e.contents[0] for e in soup.find_all('a')]
sequences = [e.contents[0] for e in soup.find_all('li')]

In [210]:
seqs = {}
for i, name in enumerate(names):
    id = 'peptaibol_'+name #creating new
    seq = sequences[i].strip(' ').strip('\n')
    rowdata = {'id':id, 'name':name, 'AB':1, 'description':'None', 'OX':'alien', 'dataset':'peptaibol.html', 'seq':seq}
    seqs[name] = rowdata

df = pd.DataFrame.from_dict(seqs, orient='index')
df.to_csv('peptaibol_out.csv', encoding='utf-8')
df.head(2)

,id,name,AB,description,OX,dataset,seq
Aibellin,peptaibol_Aibellin,Aibellin,1,None,alien,peptaibol.html,Ac U A U A U A Q U F U G U U P V U U E E NHC(C...
Alamethicin_F-30,peptaibol_Alamethicin_F-30,Alamethicin_F-30,1,None,alien,peptaibol.html,Ac U P U A U A Q U V U G L U P V U U E Q F OH


http://crdd.osdd.net/raghava/parapep/

In [211]:
url = 'http://crdd.osdd.net/raghava/parapep/submitkey_target_search.php?ran=5411'
request = requests.get(url)
soup = bs(request.text)

rows = soup.find_all("tbody")[0].find_all('tr')# 766

In [212]:
seqs = {}
for r in rows:
    aas = r.find_all('a')
    localid = aas[0].contents[0]
    pmid = aas[1].contents[0]
    tds = r.find_all('td')
    seq = [localid, pmid] + [t.contents[0] for t in tds[2:]]
    seqs[pmid] = seq

df = pd.DataFrame.from_dict(seqs, orient='index')
heads = ['id', 'pmid', 'sequence', 'name', 'cyclic/linear', 'chirality', 'chemicalmodification', 'ntermod', 'ctermod', 'origin', 'nature', 'assaytype', 'target', 'invivo/vitro', 'disease', 'activity', 'hemolysis', '3dstructrue']
df.columns = heads
df.head(2)# (101, 18)

,id,pmid,sequence,name,cyclic/linear,chirality,chemicalmodification,ntermod,ctermod,origin,nature,assaytype,target,invivo/vitro,disease,activity,hemolysis,3dstructrue
17376436,1668,17376436,ZCRRLCYKQRCVTYCRGR,Gomesin,"Cyclic (C2,C15)and (C6,C11)",L,Z=pyroglutamic acid,Free,Free,Hemocytes of the spider Acanthoscurria gomesiana,Antimicrobial,Microscopy (Neubauer hemocytometer),Permeabilization of cell membrane,In vitro,Malaria,75% inhibition at 12.5ÂµM,16% hemolysis at 1ÂµM,[[]]
20032878,1011,20032878,FOV,Ultra-short peptide 9,Linear,L,O=Ornithine,Free,Free,Synthetic,Antimalarial,Microscopy (Giemsa staining),Not defined,In vitro,Malaria,IC,Not reported,[[]]


In [213]:
dfnew = df[['pmid', 'name', 'origin', 'sequence']]
description = list( zip(df.nature.values, df['invivo/vitro'].values, df.target.values) )

dfnew.insert(2, 'AB', [1 for i in range(dfnew.shape[0]) ] )
dfnew.insert(3, 'description', description)
dfnew.insert(5, 'dataset', ['parapep.html' for i in range(dfnew.shape[0]) ] )


colnames = ['id', 'name', 'AB', 'description', 'OX', 'dataset', 'seq']
dfnew.columns = colnames
dfnew.to_csv('parapep_out.csv', encoding='utf-8')
dfnew.head(2)

,id,name,AB,description,OX,dataset,seq
17376436,17376436,Gomesin,1,"(Antimicrobial, In vitro, Permeabilization of ...",Hemocytes of the spider Acanthoscurria gomesiana,parapep.html,ZCRRLCYKQRCVTYCRGR
20032878,20032878,Ultra-short peptide 9,1,"(Antimalarial, In vitro, Not defined)",Synthetic,parapep.html,FOV


https://webs.iiitd.edu.in/raghava/antitbpdb/ 

select for peptide type = natural
https://webs.iiitd.edu.in/raghava/antitbpdb/browse_sub1.php?token=Natural&col=12

In [217]:
url = 'https://webs.iiitd.edu.in/raghava/antitbpdb/browse_sub1.php?token=Natural&col=12'
request = requests.get(url)
soup = bs(request.text)

In [221]:
header = soup.find("thead").find_all('th')# 766
colnames = [h.contents[0] for h in header]

In [231]:
rows = soup.find("tbody").find_all('tr')

In [232]:
seqs = {}
for r in rows:
    tds = r.find_all('td')
    id = tds[0].find_all('a')[0].contents[0]

    contents = [el.find_all('font')[0].contents[0] for el in tds[1:]]
    # 0 = name, 1 = seq
    contents[1] = contents[1].contents[0]
    contents = [id]+contents
    seqs[id] = contents

df = pd.DataFrame.from_dict(seqs, orient='index')
df.columns = colnames
df.head(2)

,ID,Name,Sequence,N-Terminal Modification,C-Terminal Modification,Chemical Modification,Linear/Cyclic,Length,Chirality,Nature,...,Cytotoxicity,Animal Model,Effective Dose in model organism,Immune Responce,Mechanism of Action,Target,Combination Therapy,Other Activities,Year of Publication,Pubmed ID/ Patent No.
antitb_1001,antitb_1001,Polydim-I,AVAGEKLWLLPHLLKMLLTPTP,Free,Free,None,Linear,22,L,Amphipathic,...,"Non-cytotoxic, 10% cytotoxicity on J774 cells ...",6 to 8 weeks old BALB/c and IFN-Î³KO (Knockout...,2 mg/kg/mLW shows 90% reduction in bacterial load,NA,Cell wall disruption,Cell wall,None,NA,2016,[26930596]
antitb_1002,antitb_1002,Ecumicin,"(N,N-Me2-Val)-(Val)-(N-Me-allo-Ilu)-(Thr)-(N-M...",Free,Free,"N-methylation pattern, N-Me-Val = N-methyl-val...",Macrocyclic,13,L,NA,...,>63 Î¼M for vero cells,"Female BALB/c mice, male CD-1 mice",20 mg/kg causes reductions in M. tuberculosis ...,NA,Ecumicin markedly enhanced the ATPase activity...,ClpC1 ATPase complex,Ecumicin encapsulated in micelles distributes ...,NA,2015,[25421483]


In [234]:
dfnew = df[['ID', 'Name','Animal Model', 'Sequence']]
description = list( zip( df.Nature.values, df['Mechanism of Action'].values, df.Target.values) )

dfnew.insert(2, 'AB', [1 for i in range(dfnew.shape[0]) ] )
dfnew.insert(3, 'description', description)
dfnew.insert(5, 'dataset', ['AntiTbPdb.html' for i in range(dfnew.shape[0]) ] )

colnames = ['id', 'name', 'AB', 'description', 'OX', 'dataset', 'seq']
dfnew.columns = colnames
dfnew.to_csv('AntiTbPdb_out.csv', encoding='utf-8')
dfnew.head(2)

,id,name,AB,description,OX,dataset,seq
antitb_1001,antitb_1001,Polydim-I,1,"(Amphipathic, Cell wall disruption , Cell wall)",6 to 8 weeks old BALB/c and IFN-Î³KO (Knockout...,AntiTbPdb.html,AVAGEKLWLLPHLLKMLLTPTP
antitb_1002,antitb_1002,Ecumicin,1,"(NA, Ecumicin markedly enhanced the ATPase act...","Female BALB/c mice, male CD-1 mice",AntiTbPdb.html,"(N,N-Me2-Val)-(Val)-(N-Me-allo-Ilu)-(Thr)-(N-M..."
